In [3]:
import pandas as pd
import numpy as np
import duckdb   

# Chargement des données

In [4]:
# General Path to the directory
csv_path = 'https://github.com/gunony/Example_SQL_Data/raw/main/'

# Path to the databases
path_item = csv_path + 'item.csv'
path_client = csv_path + 'client.csv'
path_invoice = csv_path + 'invoice.csv'
#path_order = csv_path + 'wine_sales_2024_Order.csv'

# Load the tables
table_item = pd.read_csv(path_item)
table_client = pd.read_csv(path_client)
table_invoice = pd.read_csv(path_invoice, parse_dates=['date'])
#table_order = pd.read_csv(path_order, parse_dates=['date'])


Regarder la composition de la table des articles.\
La clé est le code_item.

In [5]:
query = """
SELECT *
FROM table_item
"""
duckdb.sql(query)

┌────────────┬─────────────────────────────────┬───────┬────────────┬────────────────┬─────────┐
│ code_item  │            item_name            │ year  │ base_price │     region     │  type   │
│  varchar   │             varchar             │ int64 │   double   │    varchar     │ varchar │
├────────────┼─────────────────────────────────┼───────┼────────────┼────────────────┼─────────┤
│ rpo2013100 │ Château L'Evangile 2013         │  2013 │      139.0 │ Pomerol        │ rouge   │
│ res2017100 │ Château Cos d'Estournel 2017    │  2017 │      150.0 │ Saint Estèphe  │ rouge   │
│ bsa2012100 │ Château Rayne Vigneau 2012      │  2012 │      137.0 │ Sauternes      │ blanc   │
│ bsa2018100 │ Château d'Yquem 2018            │  2018 │      169.0 │ Sauternes      │ blanc   │
│ res2021101 │ Château Calon-Ségur 2021        │  2021 │      120.0 │ Saint Estèphe  │ rouge   │
│ rma2021101 │ Château Giscours 2021           │  2021 │      157.0 │ Margaux        │ rouge   │
│ rpa2019100 │ Château Mouton 

Regarder la composition de la table des clients, dont la clé est le cust_id.

In [6]:
query = """
SELECT *
FROM table_client
"""
duckdb.sql(query)

┌───────────┬───────────────────────┬─────────────┬─────────────────────────────┐
│ id_client │         town          │   country   │         client_name         │
│  varchar  │        varchar        │   varchar   │           varchar           │
├───────────┼───────────────────────┼─────────────┼─────────────────────────────┤
│ ar102     │ Buenos Aires          │ Argentine   │ Aramburu                    │
│ ch107     │ Shanghai              │ Chine       │ Taian Table                 │
│ ch109     │ Beijing               │ Chine       │ Xin Rong Ji                 │
│ ch239     │ Beijing               │ Chine       │ Chao Shang Chao             │
│ eu113     │ Chicago               │ Etats-Unis  │ Smyth                       │
│ eu115     │ New York              │ Etats-Unis  │ Le Bernardin                │
│ eu116     │ San Francisco         │ Etats-Unis  │ Atelier Crenn               │
│ eu117     │ San Diego             │ Etats-Unis  │ Addison                     │
│ fr116     │ An

Regarder la composition de la table des factures établies, dont la clé est le inv_no (numéro de facture).

In [7]:
query = """
SELECT *
FROM table_invoice
"""
duckdb.sql(query)

┌─────────────────────┬─────────┬────────────┬───────┬────────┬─────────┬──────────┐
│        date         │ cust_id │    item    │  qty  │ price  │ line_no │  inv_no  │
│    timestamp_ns     │ varchar │  varchar   │ int64 │ double │  int64  │ varchar  │
├─────────────────────┼─────────┼────────────┼───────┼────────┼─────────┼──────────┤
│ 2025-01-01 00:00:00 │ fr133   │ rpo1979100 │     3 │ 5320.0 │       1 │ fac01_01 │
│ 2025-01-03 00:00:00 │ fr141   │ rch2009100 │    10 │  210.0 │       1 │ fac01_02 │
│ 2025-01-05 00:00:00 │ fr116   │ rpo2013100 │     5 │  148.8 │       1 │ fac01_03 │
│ 2025-01-09 00:00:00 │ fr133   │ bsa2012100 │     7 │   39.7 │       1 │ fac01_04 │
│ 2025-01-10 00:00:00 │ fr140   │ bsa2018100 │     3 │  181.0 │       1 │ fac01_05 │
│ 2025-01-11 00:00:00 │ eu116   │ res2021101 │    12 │  156.0 │       1 │ fac01_06 │
│ 2025-01-11 00:00:00 │ eu116   │ rpo2013100 │    12 │ 173.75 │       2 │ fac01_06 │
│ 2025-01-15 00:00:00 │ fr124   │ rma2021101 │     4 │  168.0 │  

Regarder la table des commandes avec comme colonne clé 'order_no' qui correspond au numéro de commande.

In [8]:
query = """
SELECT *
FROM table_order
LIMIT 10 
"""
# duckdb.sql(query)

# Les Jointures
Une jointure en SQL permet de combiner des données provenant de plusieurs tables en fonction d'une relation logique entre elles.

### 1-Cross join
C'est un produit cartésien. Chaque ligne de la première table est combinée avec les lignes de la seconde table.

**Mission** : Affecter à chaque client la gamme complète des vins.

In [10]:
query = '''

SELECT
    id_client,
    client_name,
    code_item,
    item_name

FROM table_client

CROSS JOIN table_item
'''
duckdb.sql(query)

┌───────────┬─────────────────────────────┬────────────┬────────────────────────────┐
│ id_client │         client_name         │ code_item  │         item_name          │
│  varchar  │           varchar           │  varchar   │          varchar           │
├───────────┼─────────────────────────────┼────────────┼────────────────────────────┤
│ ar102     │ Aramburu                    │ rpo2013100 │ Château L'Evangile 2013    │
│ ch107     │ Taian Table                 │ rpo2013100 │ Château L'Evangile 2013    │
│ ch109     │ Xin Rong Ji                 │ rpo2013100 │ Château L'Evangile 2013    │
│ ch239     │ Chao Shang Chao             │ rpo2013100 │ Château L'Evangile 2013    │
│ eu113     │ Smyth                       │ rpo2013100 │ Château L'Evangile 2013    │
│ eu115     │ Le Bernardin                │ rpo2013100 │ Château L'Evangile 2013    │
│ eu116     │ Atelier Crenn               │ rpo2013100 │ Château L'Evangile 2013    │
│ eu117     │ Addison                     │ rpo2013100

### 2-Inner join
Cette opération permet de combiner les lignes de deux tables en se basant sur une condition de jointure spécifique.\
La jointure renvoie uniquement les lignes où il y a une correspondance dans les deux tables.\
C'est un 'crossjoin' avec une condition de correspondance.\
Attention : S'il manque des informations dans une table, alors la ligne avec cette information manquante n'apparaîtra pas.

**Mission** : Ajouter dans la table des ventes le nom des clients et le libelle des produits.

In [ ]:
query = '''
SELECT
    date,
    cust_id,
    client_name,
    item,
    item_name,
    inv_no
FROM 
    table_invoice
INNER JOIN
    table_client
ON 
    cust_id = id_client
INNER JOIN
    table_item
ON
    item = code_item
'''

duckdb.sql(query)


┌─────────────────────┬─────────┬─────────────────────────┬────────────┬─────────────────────────────────┬──────────┐
│        date         │ cust_id │       client_name       │    item    │            item_name            │  inv_no  │
│    timestamp_ns     │ varchar │         varchar         │  varchar   │             varchar             │ varchar  │
├─────────────────────┼─────────┼─────────────────────────┼────────────┼─────────────────────────────────┼──────────┤
│ 2025-01-01 00:00:00 │ fr133   │ L'Ambroisie             │ rpo1979100 │ Pétrus 1979                     │ fac01_01 │
│ 2025-01-03 00:00:00 │ fr141   │ Assiette Champenoise    │ rch2009100 │ Deutz 2009                      │ fac01_02 │
│ 2025-01-05 00:00:00 │ fr116   │ Le Clos des Sens        │ rpo2013100 │ Château L'Evangile 2013         │ fac01_03 │
│ 2025-01-09 00:00:00 │ fr133   │ L'Ambroisie             │ bsa2012100 │ Château Rayne Vigneau 2012      │ fac01_04 │
│ 2025-01-10 00:00:00 │ fr140   │ Le Pré Catelan        

### 3-Left join
C'est une combinaison des lignes de deux tables avec un condition de jointure spécifique, mais qui garde toute les lignes de la table de gauche (1ière table) et les lignes correspondantes de la table de droite (2nd table).

**Mission :** Rédiger une requête qui permet d'avoir sur la table des ventes le nom des clients et le titres des articles.

In [15]:
query ="""
SELECT 
    date,
    cust_id,
    client_name,
    item,
    item_name,
    qty,
    price,
    line_no,
    inv_no
FROM
    table_invoice
LEFT JOIN 
    table_client
    ON 
        cust_id = id_client
LEFT JOIN
    table_item
    ON
        item = code_item
"""
duckdb.sql(query)

┌─────────────────────┬─────────┬─────────────────────────┬────────────┬─────────────────────────────────┬───────┬────────┬─────────┬──────────┐
│        date         │ cust_id │       client_name       │    item    │            item_name            │  qty  │ price  │ line_no │  inv_no  │
│    timestamp_ns     │ varchar │         varchar         │  varchar   │             varchar             │ int64 │ double │  int64  │ varchar  │
├─────────────────────┼─────────┼─────────────────────────┼────────────┼─────────────────────────────────┼───────┼────────┼─────────┼──────────┤
│ 2025-01-01 00:00:00 │ fr133   │ L'Ambroisie             │ rpo1979100 │ Pétrus 1979                     │     3 │ 5320.0 │       1 │ fac01_01 │
│ 2025-01-03 00:00:00 │ fr141   │ Assiette Champenoise    │ rch2009100 │ Deutz 2009                      │    10 │  210.0 │       1 │ fac01_02 │
│ 2025-01-05 00:00:00 │ fr116   │ Le Clos des Sens        │ rpo2013100 │ Château L'Evangile 2013         │     5 │  148.8 │       

#### 4-Full outer join
C'est la combinaison des résultats d'un 'left join' et d'un 'right join'.\
Le résultat retourne toutes les lignes des deux tables avec des valeurs NULL lorsqu'il n'y a pas de correspondances.

**Mission** : Lister les clients qui n'ont pas été facturés sur la période. 


In [ ]:
query = '''
SELECT 
    id_client,
    client_name,
    town,
    country,
    cust_id
FROM
    table_invoice
FULL OUTER JOIN
    table_client
ON
    cust_id = id_client
WHERE
    cust_id IS NULL
'''
duckdb.sql(query)

┌───────────┬────────────────────────┬───────────────────────┬─────────────┬─────────┐
│ id_client │      client_name       │         town          │   country   │ cust_id │
│  varchar  │        varchar         │        varchar        │   varchar   │ varchar │
├───────────┼────────────────────────┼───────────────────────┼─────────────┼─────────┤
│ ch109     │ Xin Rong Ji            │ Beijing               │ Chine       │ NULL    │
│ eu113     │ Smyth                  │ Chicago               │ Etats-Unis  │ NULL    │
│ eu117     │ Addison                │ San Diego             │ Etats-Unis  │ NULL    │
│ fr117     │ L'Oustau de Baumanière │ Les Baux-de-Provence  │ France      │ NULL    │
│ hk221     │ Caprice                │ Hong Kong             │ Hong Kong   │ NULL    │
│ it224     │ Le Calandre            │ Rubano                │ Italie      │ NULL    │
│ ru231     │ L'Enclume              │ Cartmel               │ Royaume-Uni │ NULL    │
│ ru233     │ Fat Duck               │ Bray

### 5-Self join
C'est une opération de jointure d'une table avec elle-même.\
Cette jointure nécessite d'utiliser des alias dans les commandes SELECT, FROM et JOIN pour distinguer les deux instances.

**Mission** : Ecrire une requête qui permet d'obtenir le nombre de client facturé la semaine présente et la semaine précédente (i.e. retained client).

In [ ]:
query = """

-- créer une sous_table en provenance des ventes avec numéro client, nom et le no de la semaine--

WITH cte AS (
    SELECT 
        cust_id,
        WEEK(date) as week,
    FROM 
        table_invoice
    GROUP BY
        cust_id,
        week
    ORDER BY 
        week 
)

-- requete principale avec un SELF JOIN --
SELECT 
    l.week,
    COUNT(DISTINCT(l.cust_id)) AS retained_client
FROM 
    cte AS l
JOIN
    cte AS r
ON 
    l.cust_id = r.cust_id
    AND
    l.week = r.week + 1
GROUP BY 
    l.week
ORDER BY
    l.week

"""
duckdb.sql(query)

┌───────┬─────────────────┐
│ week  │ retained_client │
│ int64 │      int64      │
├───────┼─────────────────┤
│     2 │               1 │
│     8 │               2 │
│     9 │               1 │
│    10 │               1 │
│    13 │               1 │
│    15 │               1 │
│    16 │               2 │
│    19 │               1 │
└───────┴─────────────────┘

# Les window functions
Les fonctions fenêtres permettent d'effectuer des calculs sur un ensemble de lignes tout en conservant les lignes d'origines.

Pour rappel l'ordre des clauses est le suivant 'PARTITION BY' (divise en groupe) puis 'ORDER BY' (classe les lignes) puis 'ROWS BETWEEN' (définie les lignes dans la plage). 

### 1-La clause OVER()
Cette clause s'utilise avec un fonction d'aggrégation ou avec des fonctions comme ROW_NUMBER, RANK(), DENSE_RANK() ou LAG().

**OVER (ORDER BY...)**
Cette clause définie la manière dont les lignes d'une partition seront triées selon une colonne.\
OVER (ORDER BY expr ASC/DESC)


**Mission** : Ecrire une requête qui permet de connaitre les ventes cumulées par semaine.

In [13]:
query = """
-- Création d'un CTE pour avoir les ventes par semaine --
WITH weekly_sales AS (
    SELECT 
        WEEK(date) AS week_no,
        SUM(qty * price) AS weekly_sales
    FROM 
        table_invoice
    GROUP BY 
        WEEK(date)
)

-- Requête finale pour obtenir les ventes cumulées par semaine --
SELECT 
    week_no,
    weekly_sales,
    SUM(weekly_sales) OVER (ORDER BY week_no) AS cumulated_sales
FROM 
    weekly_sales
"""
duckdb.sql(query)


┌─────────┬───────────────────┬────────────────────┐
│ week_no │   weekly_sales    │  cumulated_sales   │
│  int64  │      double       │       double       │
├─────────┼───────────────────┼────────────────────┤
│       1 │           18804.0 │            18804.0 │
│       2 │            4777.9 │            23581.9 │
│       3 │            6595.2 │ 30177.100000000002 │
│       4 │           14364.3 │            44541.4 │
│       5 │           20904.0 │            65445.4 │
│       6 │            3961.5 │            69406.9 │
│       7 │           16705.4 │  86112.29999999999 │
│       8 │            8670.0 │  94782.29999999999 │
│       9 │           10313.5 │ 105095.79999999999 │
│      10 │            3956.9 │ 109052.69999999998 │
│      11 │            8309.2 │ 117361.89999999998 │
│      12 │            5706.4 │ 123068.29999999997 │
│      13 │            9072.0 │           132140.3 │
│      14 │          20478.65 │ 152618.94999999998 │
│      15 │ 5101.799999999999 │ 157720.7499999

**OVER (ORDER BY ... ROWS BETWEEN ...)**\
Cette commande détermine précisement quelles lignes sont à inclure dans le calcul.\
On utilise 'PRECEDING', 'FOLLOWING', 'UNBOUNDED' et 'CURRENT ROW'.

**Mission** : A partir le la table des factures calculer les ventes moyennes sur les 3 derniers jours facturées. 

In [14]:
query = '''

-- calculer les ventes journalières avec un CTE
WITH cte1 AS (
    SELECT 
        date,
        SUM(qty*price) AS sales
    FROM 
        table_invoice
    GROUP BY
        date
)

-- requête finale 
SELECT
    date,
    sales,
    AVG(sales) OVER (ORDER BY date
                            ROWS BETWEEN 3 PRECEDING and CURRENT ROW) AS average_sales_3d
FROM 
    cte1
'''
duckdb.sql(query)

┌─────────────────────┬────────────────────┬───────────────────┐
│        date         │       sales        │ average_sales_3d  │
│    timestamp_ns     │       double       │      double       │
├─────────────────────┼────────────────────┼───────────────────┤
│ 2025-01-01 00:00:00 │            15960.0 │           15960.0 │
│ 2025-01-03 00:00:00 │             2100.0 │            9030.0 │
│ 2025-01-05 00:00:00 │              744.0 │            6268.0 │
│ 2025-01-09 00:00:00 │ 277.90000000000003 │          4770.475 │
│ 2025-01-10 00:00:00 │              543.0 │           916.225 │
│ 2025-01-11 00:00:00 │             3957.0 │          1380.475 │
│ 2025-01-15 00:00:00 │              672.0 │          1362.475 │
│ 2025-01-17 00:00:00 │             5923.2 │            2773.8 │
│ 2025-01-22 00:00:00 │            14364.3 │          6229.125 │
│ 2025-01-30 00:00:00 │             2784.0 │          5935.875 │
│          ·          │                ·   │               ·   │
│          ·          │  

**OVER (PARTITION BY ...)**\
Cette clause permet de diviser les données en sous-ensembles (=partition) selon les valeurs dans les colonnes spécifiées.

**Mission** : Rédiger une requête qui donne les ventes totales par client et le total des ventes de leur pay respectif.

In [15]:
query = """

-- Création d'un CTE pour ajouter le nom du client et son pays dans la table des ventes --
WITH CTE AS (
    SELECT
        cust_id,
        client_name,
        SUM (qty * price) AS sales_cust,
        country
    FROM 
        table_invoice
    LEFT JOIN 
        table_client
    ON
        cust_id = id_client
    GROUP BY
        cust_id,
        client_name,
        country
)
SELECT 
    *,
    SUM(sales_cust) 
        OVER (PARTITION BY country) AS sales_country
FROM
    CTE
"""
duckdb.sql(query)

┌─────────┬─────────────────────────────────────────┬────────────────────┬─────────────┬───────────────┐
│ cust_id │               client_name               │     sales_cust     │   country   │ sales_country │
│ varchar │                 varchar                 │       double       │   varchar   │    double     │
├─────────┼─────────────────────────────────────────┼────────────────────┼─────────────┼───────────────┤
│ ru232   │ The ledbury                             │             1895.0 │ Royaume-Uni │        7818.2 │
│ ru236   │ Midsummer House                         │             5923.2 │ Royaume-Uni │        7818.2 │
│ ar102   │ Aramburu                                │             4830.0 │ Argentine   │        4830.0 │
│ it236   │ Enoteca Pinchiorri                      │            17278.3 │ Italie      │       39636.5 │
│ it241   │ Casa Perbellini 12 Apostoli             │             3480.0 │ Italie      │       39636.5 │
│ it223   │ Enrico Bartolini al Mudec               │  

### 2-La clause LAG

LAG (*expression* [,*decalage*[,*valeur_par_défaut*]])

La commande 'LAG' est utilisée dans un 'SELECT' avec la commande 'OVER'.\
Elle permet d'accéder à une valeur provenant d'une ligne précédente dans une colonne définie.

**Mission** : Créer une requête qui calcule les ventes réalisées d'une semaine sur l'autre avec les variations.

In [16]:
query = """
-- Créer la table des ventes par semaine --
WITH cte AS (
    SELECT
        WEEK(date) AS week_no,
        sum(qty * price) AS sales
    FROM
        table_invoice
    GROUP BY
        (week_no)
    ORDER BY
        week_no
)
-- Requête finale --
SELECT
    week_no,
    sales,
    ROUND(LAG(sales,0) OVER(ORDER BY week_no) - LAG(sales, 1) OVER(ORDER BY week_no),0) AS variance,
    ROUND((variance / LAG(sales, 1) OVER(ORDER BY week_no) * 100), 2) AS var_percent
FROM
    cte
"""

duckdb.sql(query)

┌─────────┬───────────────────┬──────────┬─────────────┐
│ week_no │       sales       │ variance │ var_percent │
│  int64  │      double       │  double  │   double    │
├─────────┼───────────────────┼──────────┼─────────────┤
│       1 │           18804.0 │     NULL │        NULL │
│       2 │            4777.9 │ -14026.0 │      -74.59 │
│       3 │            6595.2 │   1817.0 │       38.03 │
│       4 │           14364.3 │   7769.0 │       117.8 │
│       5 │           20904.0 │   6540.0 │       45.53 │
│       6 │            3961.5 │ -16943.0 │      -81.05 │
│       7 │           16705.4 │  12744.0 │       321.7 │
│       8 │            8670.0 │  -8035.0 │       -48.1 │
│       9 │           10313.5 │   1644.0 │       18.96 │
│      10 │            3956.9 │  -6357.0 │      -61.64 │
│      11 │            8309.2 │   4352.0 │      109.99 │
│      12 │            5706.4 │  -2603.0 │      -31.33 │
│      13 │            9072.0 │   3366.0 │       58.99 │
│      14 │          20478.65 │

### 3-ROW_NUMBER()
Cette clause permet de numéroter les lignes dans un ensemble de résultat. Elle s'utilise avec la clause OVER().\
Elle attribue un numéro unique à chaque ligne sans tenir compte des valeurs identiques.

**Mission** : A partir de la table des factures, je veux numéroter chque ligne de veles vins par leur prix de vente dans chque type de région.

In [37]:
query = """
SELECT *,
ROW_NUMBER() OVER(
    PARTITION BY cust_id
    ORDER BY qty ASC)
    AS classement
FROM 
    table_invoice
"""
duckdb.sql(query)

┌─────────────────────┬─────────┬────────────┬───────┬────────┬─────────┬──────────┬────────────┐
│        date         │ cust_id │    item    │  qty  │ price  │ line_no │  inv_no  │ classement │
│    timestamp_ns     │ varchar │  varchar   │ int64 │ double │  int64  │ varchar  │   int64    │
├─────────────────────┼─────────┼────────────┼───────┼────────┼─────────┼──────────┼────────────┤
│ 2025-01-15 00:00:00 │ fr124   │ rma2021101 │     4 │  168.0 │       1 │ fac01_07 │          1 │
│ 2025-02-21 00:00:00 │ fr124   │ res1957100 │     5 │  460.0 │       1 │ fac02_06 │          2 │
│ 2025-02-28 00:00:00 │ fr124   │ rju2018100 │     5 │  142.7 │       1 │ fac02_11 │          3 │
│ 2025-03-31 00:00:00 │ fr124   │ bpe2022101 │     5 │ 143.65 │       1 │ fac03_09 │          4 │
│ 2025-04-05 00:00:00 │ fr124   │ res2021101 │     5 │  130.8 │       1 │ fac04_04 │          5 │
│ 2025-01-30 00:00:00 │ eu115   │ bpe2020100 │    16 │  174.0 │       1 │ fac01_10 │          1 │
│ 2025-03-23 00:00:0

### 4- RANK() et DENSE_RANK()
Ces deux clauses qui s'utilisent avec la clause OVER() permettent, comme la clause ROW_NUMBER() de numéroter les lignes.\
Cependant RANK et DENSE_RANK ne numérotent pas de la même manière les lignes avec des valeurs identiques.
* RANK() attribue un numéro à chaque ligne mais les lignes ayant des valeurs identiques se verront attribuées le même rang et le rang suivant sera sauté.
* DENSE_RANK() fonctionne comme 'RANK()' mais ne saute pas de rang, il y a continuite des nombres.

**Mission** :

In [41]:
query = """
SELECT *,
RANK() OVER(
    PARTITION BY cust_id
    ORDER BY qty ASC)
    AS classement
FROM 
    table_invoice
"""
duckdb.sql(query)

┌─────────────────────┬─────────┬────────────┬───────┬────────┬─────────┬──────────┬────────────┐
│        date         │ cust_id │    item    │  qty  │ price  │ line_no │  inv_no  │ classement │
│    timestamp_ns     │ varchar │  varchar   │ int64 │ double │  int64  │ varchar  │   int64    │
├─────────────────────┼─────────┼────────────┼───────┼────────┼─────────┼──────────┼────────────┤
│ 2025-04-20 00:00:00 │ fr143   │ res2017100 │     5 │  185.0 │       1 │ fac04_12 │          1 │
│ 2025-01-10 00:00:00 │ fr140   │ bsa2018100 │     3 │  181.0 │       1 │ fac01_05 │          1 │
│ 2025-04-17 00:00:00 │ fr140   │ rch2009100 │     3 │  215.0 │       1 │ fac04_11 │          1 │
│ 2025-04-12 00:00:00 │ fr140   │ rpo2016100 │     6 │  325.9 │       1 │ fac04_08 │          3 │
│ 2025-02-07 00:00:00 │ it241   │ bch2015100 │     6 │  230.0 │       2 │ fac02_02 │          1 │
│ 2025-02-07 00:00:00 │ it241   │ rma2021101 │    12 │  175.0 │       1 │ fac02_02 │          2 │
│ 2025-01-15 00:00:0

In [43]:
query = """
SELECT *,
DENSE_RANK() OVER(
    PARTITION BY cust_id
    ORDER BY qty DESC)
    AS classement
FROM 
    table_invoice
"""
duckdb.sql(query)

┌─────────────────────┬─────────┬────────────┬───────┬────────┬─────────┬──────────┬────────────┐
│        date         │ cust_id │    item    │  qty  │ price  │ line_no │  inv_no  │ classement │
│    timestamp_ns     │ varchar │  varchar   │ int64 │ double │  int64  │ varchar  │   int64    │
├─────────────────────┼─────────┼────────────┼───────┼────────┼─────────┼──────────┼────────────┤
│ 2025-02-25 00:00:00 │ fr134   │ rma2021101 │     9 │  168.0 │       1 │ fac02_08 │          1 │
│ 2025-04-10 00:00:00 │ fr134   │ rpo2013100 │     7 │  151.5 │       1 │ fac04_06 │          2 │
│ 2025-04-29 00:00:00 │ fr134   │ rch2005100 │     6 │  540.0 │       1 │ fac04_16 │          3 │
│ 2025-03-05 00:00:00 │ fr134   │ rem2021100 │     5 │ 395.38 │       1 │ fac03_01 │          4 │
│ 2025-04-16 00:00:00 │ fr134   │ bch2013100 │     5 │  230.0 │       1 │ fac04_10 │          4 │
│ 2025-05-08 00:00:00 │ fr134   │ rpo1979100 │     2 │ 5310.0 │       1 │ fac05_04 │          5 │
│ 2025-01-17 00:00:0

### 5-QUALIFY
Cette clause permet de réaliser un filtre sur une colonne créée avec une 'window function'.

**Mission** : Créer une requête qui, pour chaque pays, liste le nom des clients ayant le deuxième niveau de ventes.

In [17]:
query ="""
SELECT 
    client_name,
    country,
    SUM(qty*price) AS sales,
    DENSE_RANK() OVER(PARTITION BY country ORDER BY sales DESC) AS index
FROM
    table_invoice
LEFT JOIN
    table_client
ON
    cust_id = id_client
GROUP BY
    client_name,
    country
QUALIFY
    index = 2
ORDER BY
    country
"""
duckdb.sql(query)

┌───────────────────────────┬─────────────┬─────────┬───────┐
│        client_name        │   country   │  sales  │ index │
│          varchar          │   varchar   │ double  │ int64 │
├───────────────────────────┼─────────────┼─────────┼───────┤
│ Chao Shang Chao           │ Chine       │ 16388.4 │     2 │
│ Atelier Crenn             │ Etats-Unis  │  3957.0 │     2 │
│ Arpège                    │ France      │ 19559.4 │     2 │
│ Enrico Bartolini al Mudec │ Italie      │ 10705.2 │     2 │
│ Sézanne                   │ Japon       │ 16460.4 │     2 │
│ The ledbury               │ Royaume-Uni │  1895.0 │     2 │
└───────────────────────────┴─────────────┴─────────┴───────┘

# La commande GROUP BY
Cette commande permet de grouper des résultats ayant des valeurs identiques dans une ou plusieurs colonnes. Elle s'utilise avec une fonction d'aggrégation.

### 1-GROUP BY et HAVING

La commande HAVING permet de filtrer les lignes produites par un GROUP BY. Elle remplace le WHERE qui ne peut pas être utilisé.

**Mission** : Créer une requête qui donne les ventes par ville uniquement pour le pays France.

In [18]:
query = '''
SELECT 
    town,
    country,
    ROUND(sum(qty*price),0) AS sales
FROM
    table_invoice
LEFT JOIN
    table_client
ON
    cust_id = id_client
GROUP BY
    town, country
HAVING
    country = 'France'
ORDER BY sales DESC
'''
duckdb.sql(query)

┌─────────────────────────────────────┬─────────┬─────────┐
│                town                 │ country │  sales  │
│               varchar               │ varchar │ double  │
├─────────────────────────────────────┼─────────┼─────────┤
│ Paris                               │ France  │ 44149.0 │
│ Courchevel                          │ France  │ 14240.0 │
│ Île de Noirmoutier / L'Herbaudière  │ France  │  5058.0 │
│ Reims                               │ France  │  2100.0 │
│ Annecy                              │ France  │  1640.0 │
│ Cassis                              │ France  │  1587.0 │
│ Menton                              │ France  │  1308.0 │
│ Saint-Tropez                        │ France  │   925.0 │
└─────────────────────────────────────┴─────────┴─────────┘

### 2-GROUPING SETS
Cette commande permet de réaliser des sous-groupes dans une requête comme si nous enchaînions plusieurs 'GROUP BY'.

**Mission** : Comment se répartissent les ventes par pays du client dans chaque région viticole ? Je veux aussi voir les ventes totales par région.

In [19]:
query ='''
-- CTE pour ajouter le pays (client) et la région (produit)
WITH cte AS(
    SELECT *  
    FROM 
            table_invoice AS r
    LEFT JOIN 
            table_client AS l
    ON
            r.cust_id = l.id_client
    LEFT JOIN
            table_item AS ll
    ON 
        r.item = ll.code_item
)

SELECT  region,
        country,
        SUM(qty * price) AS vente
FROM 
    cte
GROUP BY GROUPING SETS ((region, country),region)
ORDER BY region, country

'''
duckdb.sql(query)

┌───────────────┬─────────────┬────────────────────┐
│    region     │   country   │       vente        │
│    varchar    │   varchar   │       double       │
├───────────────┼─────────────┼────────────────────┤
│ Champagne     │ Chine       │             7088.4 │
│ Champagne     │ France      │            11143.0 │
│ Champagne     │ Italie      │             4069.0 │
│ Champagne     │ Japon       │             3720.0 │
│ Champagne     │ NULL        │            26020.4 │
│ Margaux       │ France      │             2184.0 │
│ Margaux       │ Italie      │             2100.0 │
│ Margaux       │ NULL        │             4284.0 │
│ Pauillac      │ Italie      │            11866.8 │
│ Pauillac      │ Japon       │ 14240.400000000001 │
│    ·          │   ·         │                ·   │
│    ·          │   ·         │                ·   │
│    ·          │   ·         │                ·   │
│ Saint Estèphe │ Japon       │             3840.0 │
│ Saint Estèphe │ NULL        │            164

### 3-ROLLUP
Cette clause utilisée avec 'GROUP BY' génère une hiérarchie d'agrégations.\
Elle calcule les résultats pour les colonnes dans l'ordre où elles sont spécifiées, en commençant par les niveaux les plus détaillés jusqu'aux totaux globaux.

**Mission** : 

### 4-CUBE
Cette clause génère tous les sous-ensembles possibles d'agrégations pour les colonnes spécifiées.

**Mission** : Obtenir dans la même requête le total des ventes par ville et par pays.

In [20]:
query = """
SELECT
    country,
    town,
    ROUND(SUM(qty * price), 0) AS sales
FROM
    table_invoice
LEFT JOIN
    table_client
ON
    cust_id = id_client
GROUP BY
    CUBE(country,town)
HAVING
    country NOT NULL -- pour éviter la duplication des villes
ORDER BY
    country, town
"""
duckdb.sql(query)

┌─────────────┬───────────────┬─────────┐
│   country   │     town      │  sales  │
│   varchar   │    varchar    │ double  │
├─────────────┼───────────────┼─────────┤
│ Argentine   │ Buenos Aires  │  4830.0 │
│ Argentine   │ NULL          │  4830.0 │
│ Chine       │ Beijing       │ 16388.0 │
│ Chine       │ Shanghai      │ 24672.0 │
│ Chine       │ NULL          │ 41060.0 │
│ Etats-Unis  │ New York      │  6190.0 │
│ Etats-Unis  │ San Francisco │  3957.0 │
│ Etats-Unis  │ NULL          │ 10147.0 │
│ France      │ Annecy        │  1640.0 │
│ France      │ Cassis        │  1587.0 │
│   ·         │  ·            │     ·   │
│   ·         │  ·            │     ·   │
│   ·         │  ·            │     ·   │
│ Italie      │ Rome          │  6193.0 │
│ Italie      │ Vérone        │  3480.0 │
│ Italie      │ NULL          │ 39637.0 │
│ Japon       │ Kyoto         │ 18120.0 │
│ Japon       │ Osaka         │  3174.0 │
│ Japon       │ Tokyo         │ 16460.0 │
│ Japon       │ NULL          │ 37

# Autres commandes


### 1-La commande CAST:
CAST(*expression* AS *type*)

Cette commande, utilisée dans un 'SELECT', permet de définir ou modifier le type des données d'une colonne.

**Mission** : Ajouter dans la table des factures une colonne avec les ventes arrondies à l'unité.

In [21]:
query = '''
SELECT 
    date,
    cust_id,
    item,
    qty,
    price,
    CAST(qty*price AS integer) AS total_sales
FROM 
    table_invoice
'''
duckdb.sql(query)

┌─────────────────────┬─────────┬────────────┬───────┬────────┬─────────────┐
│        date         │ cust_id │    item    │  qty  │ price  │ total_sales │
│    timestamp_ns     │ varchar │  varchar   │ int64 │ double │    int32    │
├─────────────────────┼─────────┼────────────┼───────┼────────┼─────────────┤
│ 2025-01-01 00:00:00 │ fr133   │ rpo1979100 │     3 │ 5320.0 │       15960 │
│ 2025-01-03 00:00:00 │ fr141   │ rch2009100 │    10 │  210.0 │        2100 │
│ 2025-01-05 00:00:00 │ fr116   │ rpo2013100 │     5 │  148.8 │         744 │
│ 2025-01-09 00:00:00 │ fr133   │ bsa2012100 │     7 │   39.7 │         278 │
│ 2025-01-10 00:00:00 │ fr140   │ bsa2018100 │     3 │  181.0 │         543 │
│ 2025-01-11 00:00:00 │ eu116   │ res2021101 │    12 │  156.0 │        1872 │
│ 2025-01-11 00:00:00 │ eu116   │ rpo2013100 │    12 │ 173.75 │        2085 │
│ 2025-01-15 00:00:00 │ fr124   │ rma2021101 │     4 │  168.0 │         672 │
│ 2025-01-17 00:00:00 │ ru236   │ rem2021100 │    12 │  430.0 │ 

### 2-CASE WHEN
CASE \
WHEN *condition 1* THEN *resultat 1*\
  WHEN *condition 2* THEN *resultat 2*\
     ELSE *resultat 3*\
     END AS *expression*

 Cette commande permet d'ajouter une logique conditionelle dans les requêtes et retourne différentes valeurs basées sur des conditions spécifiques.\
 Elle est surtout utilisée dans un 'SELECT' mais elle peut aussi être utilisée dans une aggrégation.
 
 **Mission 1** : Créer une requête qui regroupe les ventes par pays en deux zones géographiques ; 'local' (cad France) et 'export' pour les autres pays.

In [22]:
query = '''
-- ajouter le pays aux ventes et ajouter la colonne spécifique pour local / export
SELECT country, 
        SUM((qty * price)) AS sales,
        CASE 
            WHEN country = 'France' THEN 'local'
            ELSE 'export'
        END AS geo 
FROM 
        table_invoice AS r
LEFT JOIN 
        table_client AS l
ON
        r.cust_id = l.id_client

GROUP BY
         country
'''
duckdb.sql(query)

┌─────────────┬──────────┬─────────┐
│   country   │  sales   │   geo   │
│   varchar   │  double  │ varchar │
├─────────────┼──────────┼─────────┤
│ Chine       │  41060.4 │ export  │
│ France      │ 71006.25 │ local   │
│ Italie      │  39636.5 │ export  │
│ Argentine   │   4830.0 │ export  │
│ Japon       │  37754.8 │ export  │
│ Etats-Unis  │  10147.4 │ export  │
│ Royaume-Uni │   7818.2 │ export  │
└─────────────┴──────────┴─────────┘

**Mission 2** : Pour chaque client, calculer la somme de ses ventes par pays, avec les pays en colonne.

In [23]:
query = '''
SELECT cust_id, 
            SUM(
                CASE
                    WHEN country ='France' THEN qty * price
                    ELSE 0
                END) AS ventes_FR,
           SUM(
                CASE
                    WHEN country ='Italie' THEN qty * price
                    ELSE 0
                END) AS ventes_IT  ,      
            SUM(
                CASE
                    WHEN country ='Chine' THEN qty * price
                    ELSE 0
                END) AS vente_CH   ,    
           SUM(
                CASE
                    WHEN country ='Japon' THEN qty * price
                    ELSE 0
                END) AS vente_JP,
           SUM(
                CASE
                    WHEN country ='Etats-Unis' THEN qty * price
                    ELSE 0
                END) AS vente_US,
           SUM(
                CASE
                    WHEN country ='Royaume-Uni' THEN qty * price
                    ELSE 0
                END) AS vente_UK,      
            country,
FROM 
            table_invoice AS r
LEFT JOIN 
            table_client AS l
ON
            r.cust_id = l.id_client
GROUP BY 
        cust_id,
        country
'''
duckdb.sql(query)

┌─────────┬────────────────────┬───────────┬──────────┬──────────┬──────────┬──────────┬─────────────┐
│ cust_id │     ventes_FR      │ ventes_IT │ vente_CH │ vente_JP │ vente_US │ vente_UK │   country   │
│ varchar │       double       │  double   │  double  │  double  │  double  │  double  │   varchar   │
├─────────┼────────────────────┼───────────┼──────────┼──────────┼──────────┼──────────┼─────────────┤
│ fr133   │            21446.2 │       0.0 │      0.0 │      0.0 │      0.0 │      0.0 │ France      │
│ fr124   │            5057.75 │       0.0 │      0.0 │      0.0 │      0.0 │      0.0 │ France      │
│ it240   │                0.0 │    1980.0 │      0.0 │      0.0 │      0.0 │      0.0 │ Italie      │
│ ar102   │                0.0 │       0.0 │      0.0 │      0.0 │      0.0 │      0.0 │ Argentine   │
│ ja228   │                0.0 │       0.0 │      0.0 │   3174.4 │      0.0 │      0.0 │ Japon       │
│ ja227   │                0.0 │       0.0 │      0.0 │  18120.0 │      0

### 3-ROUND

ROUND (*valeur_numérique*, *nb_décimale*)

Cette commande permet d'arrondir une valeur numérique en choisissant le nombre de décimale.

**Mission** : Calculer les ventes par facture avec une décimale.

In [24]:
query = """
SELECT inv_no,
    cust_id,
    ROUND(SUM(qty*price),1) AS total_sales
FROM 
    table_invoice
GROUP
    BY inv_no,
    cust_id
ORDER BY 
    inv_no
"""
duckdb.sql(query)

┌──────────┬─────────┬─────────────┐
│  inv_no  │ cust_id │ total_sales │
│ varchar  │ varchar │   double    │
├──────────┼─────────┼─────────────┤
│ fac01_01 │ fr133   │     15960.0 │
│ fac01_02 │ fr141   │      2100.0 │
│ fac01_03 │ fr116   │       744.0 │
│ fac01_04 │ fr133   │       277.9 │
│ fac01_05 │ fr140   │       543.0 │
│ fac01_06 │ eu116   │      3957.0 │
│ fac01_07 │ fr124   │       672.0 │
│ fac01_08 │ ru236   │      5923.2 │
│ fac01_09 │ it236   │     14364.3 │
│ fac01_10 │ eu115   │      2784.0 │
│    ·     │   ·     │         ·   │
│    ·     │   ·     │         ·   │
│    ·     │   ·     │         ·   │
│ fac04_11 │ fr140   │       645.0 │
│ fac04_12 │ fr143   │       925.0 │
│ fac04_13 │ ar102   │      4830.0 │
│ fac04_14 │ it236   │      1375.0 │
│ fac04_15 │ ja228   │      3174.4 │
│ fac04_16 │ fr134   │      3240.0 │
│ fac05_01 │ ch239   │     16388.4 │
│ fac05_02 │ fr121   │      2868.0 │
│ fac05_03 │ it236   │      1539.0 │
│ fac05_04 │ fr134   │     10620.0 │
├

### 4-LIKE
La commande LIKE est utilisée avec WHERE et permet de sélectionner des valeurs avec les caractères spéciaux '%' qui remplacent n'importe quel nombre de caractères et '_' qui représente un seul caractère. 

**Mission** : Dans la table des produits, combien de vin de château d'Yquem d'année différente est-il possible de vendre ?

In [25]:
query = """
SELECT *
FROM table_item
WHERE item_name LIKE '%Yquem%'
"""
duckdb.sql(query)

┌────────────┬──────────────────────┬───────┬────────────┬───────────┬─────────┐
│ code_item  │      item_name       │ year  │ base_price │  region   │  type   │
│  varchar   │       varchar        │ int64 │   double   │  varchar  │ varchar │
├────────────┼──────────────────────┼───────┼────────────┼───────────┼─────────┤
│ bsa2018100 │ Château d'Yquem 2018 │  2018 │      169.0 │ Sauternes │ blanc   │
│ bsa2019100 │ Château d'Yquem 2019 │  2019 │      390.0 │ Sauternes │ blanc   │
│ bsa2020100 │ Château d'Yquem 2020 │  2020 │      430.0 │ Sauternes │ blanc   │
└────────────┴──────────────────────┴───────┴────────────┴───────────┴─────────┘

### 5-SELECT
Cette commande permet de selectionner les colonnes.
* SELECT col1, col2 : permet de sélectionner des colonnes spécifiques.
* SELECT * : sélectionne tous les colonnes de la table. A éviter.
* SELECT DISTINCT : permet de lister individuellement les données d'une colonne de la table. Cela évite les redondances.

**Mission** : Lister les région et types de vin du catalogue.

In [42]:
query="""
SELECT DISTINCT(type, region) AS type_and_region
FROM table_item
ORDER BY type

"""
duckdb.sql(query)

┌──────────────────────────┐
│     type_and_region      │
│ struct(varchar, varchar) │
├──────────────────────────┤
│ (blanc, Champagne)       │
│ (blanc, Sauternes)       │
│ (blanc, Pessac Leognan)  │
│ (rosé, Champagne)        │
│ (rouge, Saint Emilion)   │
│ (rouge, Pauillac)        │
│ (rouge, Pomerol)         │
│ (rouge, Margaux)         │
│ (rouge, Saint Julien)    │
│ (rouge, Saint Estèphe)   │
├──────────────────────────┤
│         10 rows          │
└──────────────────────────┘

### 6-COALESCE
Cette clause utilisée avec 'SELECT' permet de retourner la première valeur non nulle parmi une liste de valeurs.

**Mission** :

### 7-UNION et UNION ALL
Ces deux clauses permettent de combiner les résultats de plusieurs requête en un seul résultat. Les requêtes doivent avoir le même nombre de colonnes et les mêmes types de données.\
La clause UNION supprime les doublons à la différence de UNION ALL qui les conserve.

**Mission** :

### 8-COUNT()
Cette clause permet de compter le nombre de lignes non NULL dans une colonne.

**Mission** : Créer une requête pour connaître le nombre de clients, de pays et de villes dans la base clients.

In [38]:
query = """
SELECT 
    COUNT(DISTINCT id_client) AS nb_client,
    COUNT(DISTINCT country) AS nb_country,
    COUNT(DISTINCT town) AS nb_town
FROM
    table_client
"""
duckdb.sql(query)

┌───────────┬────────────┬─────────┐
│ nb_client │ nb_country │ nb_town │
│   int64   │   int64    │  int64  │
├───────────┼────────────┼─────────┤
│        33 │          8 │      30 │
└───────────┴────────────┴─────────┘

### 9-CTE


### 10-Ordre des clauses SQL
L'orde des réalisation des clauses SQL est le suivant :\
FROM > JOIN > WHERE > GROUP BY > HAVING > SELECT > DISTINCT > ORDER BY > LIMIT > OFFSET

### 12-FILTER
Cette cmd permet de filtrer les valeurs sur lesquelles l'aggrégation est effectuée.

**Mission** : Créer une requête qui détermine, pour chaque type de vin, le volume total des bouteilles vendues ainsi que le volume des bouteilles dont le millésime est antérieur à 2010.

In [28]:
query ="""
SELECT 
    type,
    SUM(qty) FILTER (year<2010) AS Qty_before_Yr2010,
    SUM(qty) AS All_Qty
FROM
    table_invoice
LEFT JOIN
    table_item
ON
    item = code_item
GROUP BY
    type
ORDER BY
    type
"""
duckdb.sql(query)

┌─────────┬───────────────────┬─────────┐
│  type   │ Qty_before_Yr2010 │ All_Qty │
│ varchar │      int128       │ int128  │
├─────────┼───────────────────┼─────────┤
│ blanc   │                12 │     172 │
│ rosé    │                36 │      36 │
│ rouge   │                22 │     284 │
└─────────┴───────────────────┴─────────┘

***************************************************************************************************

# 2-Solutions pratiques avec les clauses 'join'

### 2.1 - Variation hebdomadaire en pourcentage des clients avec 'Self-Join'.
Contexte :\
Évaluer l'évolution des clients est essentiel pour suivre un indicateur clé comme le nombre de clients actifs.\
Mission :\
Déterminer la variation en pourcentage d'une semaine à l'autre pour les clients hebdomadaire actifs.

Pour rappel :\
Le 'self join' est une opération de jointure d'une table avec elle-même qui nécessite d'utiliser des alias dans 'SELECT', 'FROM' et 'JOIN' pour distinguer les deux instances.


In [29]:
query = """

-- Créer un CTE avec par facture la semaine et le code client --
WITH cte AS (
    SELECT 
        WEEK(date) AS week,
        cust_id,
        inv_no
    FROM table_invoice
    GROUP BY date, cust_id, inv_no
    ORDER BY week
),
    cte2 AS (
    SELECT 
        week,
        COUNT(DISTINCT(cust_id)) AS nb_cust
    FROM cte
    GROUP BY(week)
)

-- Requête finale à l'aide de cte --

SELECT 
    l.week AS previous_week,
    l.nb_cust AS prev_nb_cust,
    r.week AS current_week,
    r.nb_cust AS curr_nb_cust,
    ROUND(100*((curr_nb_cust / prev_nb_cust)-1),2) AS percent_var_nb_cust
FROM
    cte2 AS l
JOIN
    cte2 AS r
ON 
    l.week = r.week + 1

ORDER BY previous_week

"""
duckdb.sql(query)

┌───────────────┬──────────────┬──────────────┬──────────────┬─────────────────────┐
│ previous_week │ prev_nb_cust │ current_week │ curr_nb_cust │ percent_var_nb_cust │
│     int64     │    int64     │    int64     │    int64     │       double        │
├───────────────┼──────────────┼──────────────┼──────────────┼─────────────────────┤
│             2 │            3 │            1 │            3 │                 0.0 │
│             3 │            2 │            2 │            3 │                50.0 │
│             4 │            1 │            3 │            2 │               100.0 │
│             5 │            2 │            4 │            1 │               -50.0 │
│             6 │            2 │            5 │            2 │                 0.0 │
│             7 │            2 │            6 │            2 │                 0.0 │
│             8 │            3 │            7 │            2 │              -33.33 │
│             9 │            4 │            8 │            3 │   

### 2.2 - Produits jamais commandés avec 'left join' et 'cross join':
Mission : \
Lister pour chaque client les produits du catalogue qui n'ont pas été commandés sur l'année.

Pour rappel :\
Un 'cross join' est un produit cartésien. Chaque ligne de la première table est combinée avec les lignes de la seconde table.\
Un 'left join' est une combinaison des lignes de deux tables avec un condition de jointure sécifique. La jointure garde toute les lignes de la table de gauche (1ière table) et les lignes correspondantes de la table de droite (2nd table).

In [30]:
query = """

-- définir une table des ventes par client et articles --
WITH cte_ventes AS(
    SELECT 
        cust_id,
        item,
        SUM(qty * price) AS annual_sales
    FROM
        table_invoice
    GROUP BY
        cust_id, item        
)

-- Requete principale qui liste les clients et les produits jamais commandés --

SELECT 
    l.cust_id AS cust_code,
    l.name AS cust_name, 
    r.code_item AS item_code,
    r.item_name AS item_name

FROM 
    table_client AS l
    -- permet de créer une table de tous les clients avec tous les produits --
    CROSS JOIN 
    table_item AS r
LEFT JOIN
    cte_ventes AS v
ON
    (l.cust_id = v.cust_id) AND(r.code_item = v.item) 
GROUP BY 
    l.cust_id,l.name, r.code_item, r.item_name
HAVING 
    SUM(annual_sales) IS NULL
ORDER BY
    l.cust_id, r.code_item
"""
duckdb.sql(query)

BinderException: Binder Error: Table "l" does not have a column named "cust_id"

### 2.3 - Produits jamais commandés avec 'cross join' et 'full outer join':
Meme objectif que précédement avec un FULL OUTER JOIN.

Pour rappel :\
Le résultat d'un 'full outer join' retourne toutes les lignes des deux tables avec des valeurs NULL là où il n'y a pas de correspondances.

In [ ]:
query = """
-- premier cte pour récuperer les clients et les produits effectivement vendus --
WITH 
cte1 AS (
    SELECT 
        cust_id,
        item,
    FROM 
        table_invoice
    GROUP BY 
        cust_id,
        item
    ORDER BY
        cust_id,
        item
),

-- second cte pour créer une liste des clients avec tous les produits --
cte2 AS (
    SELECT 
        cust_id,
        code_item,
        name,
        item_name
    FROM 
        table_client
    CROSS JOIN 
        table_item
    ORDER BY
        cust_id,
        code_item
)

-- on réalise la jointure des deux cte en selectionnant les clients et produits absent du premier cte --
    SELECT
        l.cust_id,
        l.name,
        l.code_item,
        l.item_name
    FROM
        cte2 AS l
    FULL OUTER JOIN
        cte1 AS r
    ON
        (l.cust_id = r.cust_id)
         AND 
        (l.code_item = r.item)
    WHERE
        r.cust_id IS NULL
    ORDER BY
        l.cust_id,
        l.code_item
"""

duckdb.sql(query)

BinderException: Binder Error: Referenced column "cust_id" not found in FROM clause!
Candidate bindings: "table_client.country", "table_item.code_item", "table_item.base_price", "table_client.client_name", "table_item.item_name"

### 2.4 - Les ventes par pays et par type de vin avec 'inner join'
Contexte :\
On cherche à savoir si il existe une relation entre le pays du client et la région du vin de bordeaux.\
Mission :\
Extraire les région de bordeaux vendu pour chaque pays de vente et le nombre de bouteilles.

Pour rappel:\
Un 'inner join' permet de combiner les lignes de deux tables en se basant sur une condition de jointure spécifique. Il renvoie uniquement les lignes où il y a une correspondance dans les deux tables. C'est un 'crossjoin' avec une condition de correspondance.

In [ ]:
query = """
SELECT 
    r.country,
    rr.region,
    SUM(l.qty) AS total_qty
FROM
    table_invoice AS l
INNER JOIN
    table_client AS r
INNER JOIN
    table_item AS rr
ON
    l.cust_id = r.cust_id
ON
    l.item = rr.code_item
GROUP BY
    r.country,
    rr.region
Order BY 
    rr.region,
    total_qty DESC
"""
duckdb.sql(query)

BinderException: Binder Error: Referenced table "r" not found!
Candidate tables: "l"

### 2.5- Commandes clients maintenues ou non d'un mois sur l'autre avec un 'self join' et un 'full ouetr join'
Mission n°1 :\
Écrire une requête qui permet d'obtenir le nombre de clients qui ont passé une commande le mois précédent et le mois présent.

In [ ]:
query = """

-- créer une sous_table en provenance des ventes avec numéro client, nom et le mois de la facture--

WITH cte AS (
    SELECT 
        cust_id,
        MONTH(date) as month,
    FROM table_invoice
    GROUP BY cust_id, month
)

-- requete principale avec un SELF JOIN --
SELECT 
    l.month,
    COUNT(DISTINCT(l.cust_id)) AS retained_client
FROM 
    cte AS l
JOIN
    cte AS r
ON 
    l.cust_id = r.cust_id
    AND
    l.month = r.month + 1
GROUP BY 
    l.month
ORDER BY
    l.month

"""
duckdb.sql(query)

┌───────┬─────────────────┐
│ month │ retained_client │
│ int64 │      int64      │
├───────┼─────────────────┤
│     2 │               2 │
│     3 │               3 │
│     4 │               3 │
│     5 │               2 │
└───────┴─────────────────┘

Mission n°2 :\
Rédiger une requête qui donne le nombre de clients du mois dernier qui n'ont pas achetés ce mois-ci.

In [ ]:
query = """

-- créer une sous_table en provenance des ventes avec numéro client, nom et le mois de la facture--

WITH cte AS (
    SELECT 
        cust_id,
        MONTH(date) as month,
    FROM table_invoice
    GROUP BY cust_id, month
)

SELECT
    l.month,
    COUNT(DISTINCT(r.cust_id)) AS left_client
FROM
    cte AS l
FULL OUTER JOIN
    cte AS r
ON
    l.cust_id = r.cust_id
    AND 
    l.month = r.month + 1
WHERE 
    l.cust_id IS NULL
GROUP BY 
    l.month
ORDER BY 
    l.month

"""
duckdb.sql(query)